In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext()
spark = SparkSession(sc)

In [15]:
df = spark.createDataFrame([(33,"known",101,"un-employed"),(32,"unknown",100,"salary"),(34,"primary",200,"business"),(35,"higher",300,"self-employeed"),(36,"primary",200,"business"),(30,"primary",200,"business"),(37,"higher",300
,"self-employeed"),(37,"postgraduate",300,"self-employeed")],["age","education","salary","employment_type"])

In [6]:
df.show()

+---+------------+------+---------------+
|age|   education|salary|employment_type|
+---+------------+------+---------------+
| 33|       known|   101|    un-employed|
| 32|     unknown|   100|         salary|
| 34|     primary|   200|       business|
| 35|      higher|   300| self-employeed|
| 36|     primary|   200|       business|
| 30|     primary|   200|       business|
| 37|      higher|   300| self-employeed|
| 37|postgraduate|   300| self-employeed|
+---+------------+------+---------------+



In [7]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

In [16]:
edu_index = StringIndexer(inputCol="education",outputCol="educationindex",handleInvalid="skip",
                          stringOrderType="alphabetDesc")#frequencyDesc"
emp_index = StringIndexer(inputCol="employment_type",outputCol="empindex",handleInvalid="error",
                          stringOrderType="alphabetDesc")
edu_index_fit =edu_index.fit(df)
df = edu_index_fit.transform(df)
emp_index_fit = emp_index.fit(df)
df = emp_index_fit.transform(df)


In [9]:
df.show()

+---+------------+------+---------------+--------------+--------+
|age|   education|salary|employment_type|educationindex|empindex|
+---+------------+------+---------------+--------------+--------+
| 33|       known|   101|    un-employed|           3.0|     0.0|
| 32|     unknown|   100|         salary|           0.0|     2.0|
| 34|     primary|   200|       business|           1.0|     3.0|
| 35|      higher|   300| self-employeed|           4.0|     1.0|
| 36|     primary|   200|       business|           1.0|     3.0|
| 30|     primary|   200|       business|           1.0|     3.0|
| 37|      higher|   300| self-employeed|           4.0|     1.0|
| 37|postgraduate|   300| self-employeed|           2.0|     1.0|
+---+------------+------+---------------+--------------+--------+



In [10]:
df_test = spark.createDataFrame([(32,"basic",100,"salary"),(35,"higher",300,"self-employeed"),(37,"higher",300
,"self-employeed"),(37,"graduate",300,"self-employeed"),(37,"primary",300,"self-employeed"),(37,"postgraduate",300,"self-employeed")],["age","education","salary","employment_type"])


df_test.show()


#edu_index.fit(df).transform(df_test).show()


#emp_index.setHandleInvalid("skip").fit(df).transform(df_test).show()


+---+------------+------+---------------+
|age|   education|salary|employment_type|
+---+------------+------+---------------+
| 32|       basic|   100|         salary|
| 35|      higher|   300| self-employeed|
| 37|      higher|   300| self-employeed|
| 37|    graduate|   300| self-employeed|
| 37|     primary|   300| self-employeed|
| 37|postgraduate|   300| self-employeed|
+---+------------+------+---------------+



In [13]:
edu_index = StringIndexer(inputCol="education",outputCol="educationindex",handleInvalid="error",
                          stringOrderType="alphabetDesc")
edu_index_fit =edu_index.fit(df)

In [14]:
edu_index_fit.transform(df_test).show()

Py4JJavaError: An error occurred while calling o221.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 4 times, most recent failure: Lost task 0.3 in stage 12.0 (TID 21, gpu3.insofe.edu.in, executor 1): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:253)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:836)
	at org.apache.spark.rdd.RDD$$anon

In [17]:
col_vec = "education_"+"vec"
col_idx = "educationindex"
encoder1 = OneHotEncoder(inputCol=col_idx,outputCol=col_vec)
df_tf2 = encoder1.transform(df)
col_vec = "emp_vec"
col_idx = "empindex"
encoder2 = OneHotEncoder(inputCol=col_idx,outputCol=col_vec)
df_tf2 = encoder2.transform(df_tf2)

In [18]:
df_tf2.show()

+---+------------+------+---------------+--------------+--------+-------------+-------------+
|age|   education|salary|employment_type|educationindex|empindex|education_vec|      emp_vec|
+---+------------+------+---------------+--------------+--------+-------------+-------------+
| 33|       known|   101|    un-employed|           3.0|     0.0|(4,[3],[1.0])|(3,[0],[1.0])|
| 32|     unknown|   100|         salary|           0.0|     2.0|(4,[0],[1.0])|(3,[2],[1.0])|
| 34|     primary|   200|       business|           1.0|     3.0|(4,[1],[1.0])|    (3,[],[])|
| 35|      higher|   300| self-employeed|           4.0|     1.0|    (4,[],[])|(3,[1],[1.0])|
| 36|     primary|   200|       business|           1.0|     3.0|(4,[1],[1.0])|    (3,[],[])|
| 30|     primary|   200|       business|           1.0|     3.0|(4,[1],[1.0])|    (3,[],[])|
| 37|      higher|   300| self-employeed|           4.0|     1.0|    (4,[],[])|(3,[1],[1.0])|
| 37|postgraduate|   300| self-employeed|           2.0|    

In [19]:
mod_features = ['age','salary','education_vec','emp_vec']
assembler = VectorAssembler(inputCols = mod_features,outputCol="features")
df_tf2 = assembler.transform(df_tf2)


In [20]:
df_tf2.show(20,False)

+---+------------+------+---------------+--------------+--------+-------------+-------------+----------------------------------+
|age|education   |salary|employment_type|educationindex|empindex|education_vec|emp_vec      |features                          |
+---+------------+------+---------------+--------------+--------+-------------+-------------+----------------------------------+
|33 |known       |101   |un-employed    |3.0           |0.0     |(4,[3],[1.0])|(3,[0],[1.0])|(9,[0,1,5,6],[33.0,101.0,1.0,1.0])|
|32 |unknown     |100   |salary         |0.0           |2.0     |(4,[0],[1.0])|(3,[2],[1.0])|(9,[0,1,2,8],[32.0,100.0,1.0,1.0])|
|34 |primary     |200   |business       |1.0           |3.0     |(4,[1],[1.0])|(3,[],[])    |(9,[0,1,3],[34.0,200.0,1.0])      |
|35 |higher      |300   |self-employeed |4.0           |1.0     |(4,[],[])    |(3,[1],[1.0])|(9,[0,1,7],[35.0,300.0,1.0])      |
|36 |primary     |200   |business       |1.0           |3.0     |(4,[1],[1.0])|(3,[],[])    |(9,[